Exception code testing

In [1]:
import sys

class CustomException(Exception):

    def __init__(self, error_message, error_details:sys):
        self.error_message = error_message
        _, _, exc_tb = error_details.exc_info()
        self.lineno = exc_tb.tb_lineno
        self.file_name = exc_tb.tb_frame.f_code.co_filename

    def __str__(self):
        return "Error occured in python script name [{0}] line number [{1}] error message [{2}]".format(
            self.file_name, self.lineno, str(self.error_message))


try:
    1 / 0
except Exception as e:
    raise CustomException(e, sys)

CustomException: Error occured in python script name [C:\Users\vchar\AppData\Local\Temp\ipykernel_36868\4159084028.py] line number [17] error message [division by zero]

Logging testing

In [2]:
import logging
import os
from datetime import datetime as dt

LOG_FILE = f"{dt.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

log_path = os.path.join(os.getcwd(), "logs")

os.makedirs(log_path, exist_ok=True)

LOG_FILEPATH = os.path.join(log_path, LOG_FILE)

logging.basicConfig(
    level=logging.INFO,
    filename=LOG_FILEPATH,
    format="[%(asctime)s] %(lineno)d %(name)s - %(levelname)s - %(message)s"
)

logging.info("Log testing executed!!!")